<a href="https://colab.research.google.com/github/chungntu/1DCNN-LSTM-ResNet/blob/main/Problem_4_First_2D_truss_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np

# ============================================================
# FEM Giàn 2D - Chỉ tính toán (Không vẽ hình)
# ============================================================

def formStiffness2Dtruss(GDof, numberElements, elementNodes, nodeCoordinates, xx, yy, EA):
    K = np.zeros((GDof, GDof), dtype=float)

    for e in range(numberElements):
        n1, n2 = elementNodes[e, 0], elementNodes[e, 1]  # ID nút bắt đầu từ 1

        # Ánh xạ bậc tự do (DOF) phần tử: [n1*2-1, n1*2, n2*2-1, n2*2] -> chỉ số bắt đầu từ 0
        edof = np.array([n1*2-1, n1*2, n2*2-1, n2*2], dtype=int) - 1

        xa = xx[n2-1] - xx[n1-1]
        ya = yy[n2-1] - yy[n1-1]
        L = np.sqrt(xa*xa + ya*ya)

        C = xa / L
        S = ya / L

        k1 = (EA / L) * np.array([
            [ C*C,  C*S, -C*C, -C*S],
            [ C*S,  S*S, -C*S, -S*S],
            [-C*C, -C*S,  C*C,  C*S],
            [-C*S, -S*S,  C*S,  S*S],
        ], dtype=float)

        K[np.ix_(edof, edof)] += k1

    return K


def solution(GDof, prescribedDof, stiffness, force, u_prescribed=None):
    if u_prescribed is None:
        u_prescribed = np.zeros(len(prescribedDof), dtype=float)

    prescribed = np.array([d-1 for d in prescribedDof], dtype=int)  # Chỉ số bắt đầu từ 0
    all_dofs = np.arange(GDof, dtype=int)
    free = np.setdiff1d(all_dofs, prescribed)

    U = np.zeros(GDof, dtype=float)
    U[prescribed] = u_prescribed

    K_ff = stiffness[np.ix_(free, free)]
    K_fp = stiffness[np.ix_(free, prescribed)]
    F_f = force[free]

    rhs = F_f - K_fp @ U[prescribed]
    U[free] = np.linalg.solve(K_ff, rhs)

    return U


def calculate_results(numberElements, elementNodes, xx, yy, displacements, E, A):
    """
    Tính toán Ứng suất (Stress) và Lực dọc (Axial Force)
    """
    sigma = np.zeros(numberElements, dtype=float)
    axial_forces = np.zeros(numberElements, dtype=float)

    print("\n=== KẾT QUẢ PHẦN TỬ (ỨNG SUẤT & LỰC DỌC) ===")
    print(f"{'PTử':<5} | {'Độ dài':<10} | {'Ứng suất (sigma)':<15} | {'Lực dọc (N)':<15} | {'Loại'}")
    print("-" * 75)

    for e in range(numberElements):
        n1, n2 = elementNodes[e, 0], elementNodes[e, 1]
        edof = np.array([n1*2-1, n1*2, n2*2-1, n2*2], dtype=int) - 1

        xa = xx[n2-1] - xx[n1-1]
        ya = yy[n2-1] - yy[n1-1]
        L = np.sqrt(xa*xa + ya*ya)

        C = xa / L
        S = ya / L

        u_e = displacements[edof]

        # Công thức tính ứng suất: E/L * [-C -S C S] * u
        sigma[e] = (E / L) * np.array([-C, -S, C, S], dtype=float) @ u_e

        # Công thức tính lực dọc: N = sigma * A
        axial_forces[e] = sigma[e] * A

        # Xác định trạng thái Kéo (+) hay Nén (-)
        # Dùng ngưỡng nhỏ 1e-9 để tránh lỗi hiển thị số 0
        if axial_forces[e] > 1e-9:
            state = "Kéo (+)"
        elif axial_forces[e] < -1e-9:
            state = "Nén (-)"
        else:
            state = "Không lực"

        print(f"{e+1:<5} | {L:<10.2f} | {sigma[e]:<15.4f} | {axial_forces[e]:<15.4f} | {state}")

    return sigma, axial_forces


def outputDisplacementsReactions(displacements, stiffness, GDof, prescribedDof, force):
    R = stiffness @ displacements - force

    print("\n=== CHUYỂN VỊ (TẠI CÁC NÚT) ===")
    for i in range(GDof):
        val = displacements[i]
        # Làm sạch số 0 âm hoặc số quá nhỏ
        if abs(val) < 1e-9: val = 0.0
        print(f"DOF {i+1}: U = {val:.6f}")

    print("\n=== PHẢN LỰC (TẠI LIÊN KẾT) ===")
    for d in prescribedDof:
        val = R[d-1]
        if abs(val) < 1e-9: val = 0.0
        print(f"DOF {d}: R = {val:.6f}")


# ============================================================
# Chương trình chính (Main Script)
# ============================================================

# 1. Vật liệu và Tiết diện
E = 30e6  # Mô đun đàn hồi (Modulus of Elasticity)
A = 2.0   # Diện tích mặt cắt ngang (Cross-sectional Area)
EA = E * A

# 2. Lưới phần tử (Mesh)
numberElements = 3
numberNodes = 4

elementNodes = np.array([
    [1, 2], # Phần tử 1 nối nút 1-2
    [1, 3], # Phần tử 2 nối nút 1-3
    [1, 4]  # Phần tử 3 nối nút 1-4
], dtype=int)

nodeCoordinates = np.array([
    [0.0,   0.0],   # Nút 1
    [0.0, 120.0],   # Nút 2
    [120.0, 120.0], # Nút 3
    [120.0, 0.0]    # Nút 4
], dtype=float)

xx = nodeCoordinates[:, 0]
yy = nodeCoordinates[:, 1]

# 3. Khởi tạo bậc tự do (DOFs)
GDof = 2 * numberNodes
displacements = np.zeros(GDof, dtype=float)
force = np.zeros(GDof, dtype=float)

# 4. Tải trọng tác dụng (Applied Load)
force[1] = -10000.0

# 5. Tính ma trận độ cứng (Stiffness Matrix)
stiffness = formStiffness2Dtruss(
    GDof, numberElements, elementNodes, nodeCoordinates, xx, yy, EA
)

# 6. Điều kiện biên (Boundary Conditions)
# Các nút 2, 3, 4 bị ngàm chặt (cả x và y đều = 0)
# Nút 2: DOF 3,4 | Nút 3: DOF 5,6 | Nút 4: DOF 7,8
prescribedDof = list(range(3, 9))

# 7. Giải hệ phương trình (Solution)
displacements = solution(GDof, prescribedDof, stiffness, force)

# 8. Xuất kết quả
outputDisplacementsReactions(displacements, stiffness, GDof, prescribedDof, force)
sigma, axial_forces = calculate_results(numberElements, elementNodes, xx, yy, displacements, E, A)


=== CHUYỂN VỊ (TẠI CÁC NÚT) ===
DOF 1: U = 0.004142
DOF 2: U = -0.015858
DOF 3: U = 0.000000
DOF 4: U = 0.000000
DOF 5: U = 0.000000
DOF 6: U = 0.000000
DOF 7: U = 0.000000
DOF 8: U = 0.000000

=== PHẢN LỰC (TẠI LIÊN KẾT) ===
DOF 3: R = 0.000000
DOF 4: R = 7928.932188
DOF 5: R = 2071.067812
DOF 6: R = 2071.067812
DOF 7: R = -2071.067812
DOF 8: R = 0.000000

=== KẾT QUẢ PHẦN TỬ (ỨNG SUẤT & LỰC DỌC) ===
PTử   | Độ dài     | Ứng suất (sigma) | Lực dọc (N)     | Loại
---------------------------------------------------------------------------
1     | 120.00     | 3964.4661       | 7928.9322       | Kéo (+)
2     | 169.71     | 1464.4661       | 2928.9322       | Kéo (+)
3     | 120.00     | -1035.5339      | -2071.0678      | Nén (-)
